## Context Managers

- When opening files or acquiring locks, resources must be released even if errors occur.
- Manual try...finally ensures cleanup but adds boilerplate and potential for mistakes.

In [ ]:
f = None

try:
    f = open("my_log.txt", "w")
    f.write("First line\n")
    f.write("Second line\n")
except:
    print("Error has occurred.")
finally:
    if f:
        print("Closing file.")
        f.close()

print(f"File closed: {f.closed}")

Closing file.
File closed: True


- Files: with open(...) as f: for automatic file closing.
- Locks: with threading.Lock(): acquires and releases locks safely.
- Tempfiles/Dirs: with tempfile.TemporaryDirectory() as d: creates and cleans up temporary directories.
- Context managers from the standard library cover most resource-management needs.

In [3]:
f = None

try:
    with open("my_log.txt", "w") as f:
        f.write("First line\n")
        f.write("Second line\n")
except:
    print("Error has occurred.")

print(f"File closed: {f.closed}")

File closed: True


In [5]:
import tempfile, os

dir_name = None

with tempfile.TemporaryDirectory() as tempdir:
    print(f"Temporary directory created: {tempdir}")
    
    dir_name = tempdir
    test_file = os.path.join(tempdir, "test.txt")
    with open(test_file, "w") as f:
        f.write("This is a test file.\n")
        
    print(f"Test file created: {os.listdir(tempdir)}")
    
try:
    content = os.listdir(dir_name)
    print(f"Temporary directory {dir_name} still exists: {content}")

except FileNotFoundError as e:
    print(f"Temporary directory {dir_name} has been cleaned up: {e}")
    

Temporary directory created: /tmp/tmpguceyxtn
Test file created: ['test.txt']
Temporary directory /tmp/tmpguceyxtn has been cleaned up: [Errno 2] No such file or directory: '/tmp/tmpguceyxtn'


## Custom Resource Management: Writing Context Managers

- Whenever you need custom setup/teardown logic, you can write your own Context Manager.
- A context manager ensures that teardown always runs, even if errors occur in the block.
- Two approaches: implement __enter__/__exit__ in a class or use the simpler generator-based decorator

In [ ]:
class MyContextManager:
    def __init__(self, timeout):
        self.timeout = timeout

    def __enter__(self):
        print("Setup complete")
        return "a simple value"

    def __exit__(self, exception_type, exception_value, traceback):
        print(f"Teardown")

        return True

with MyContextManager(timeout=30) as cm:
    print(cm) # assgn a simple value
    print("Inside the block")
    raise ValueError("Simulated problem")

Setup complete
a simple value
Inside the block
Teardown
